In [1]:
import numpy as np
import pandas as pd

In [2]:
from bs4 import BeautifulSoup

In [3]:
import requests
import time

In [4]:
url = 'https://www.chinatimes.com/search/韓國瑜?chdtv&page'

In [5]:
res = requests.get(url)
# print(res.text) #查看抓到的原始碼的長相 (因為很長，這邊就不列出來了)

In [6]:
soup = BeautifulSoup(res.text, 'lxml')

In [7]:
count = soup.find('span',{'class':'search-result-count'}).get_text().replace(',','') #看有幾筆資料
print(count)

18035


In [10]:
df = pd.DataFrame({'title':[0] ,'context': [0],'url':[0] }) #創一個表格
print(df)

   title  context  url
0      0        0    0


In [11]:
for i in range(0,10): #判斷總頁數有多少可以用：int(np.ceil(int(count)/20)) (np.ceil是無條件進位)，先判斷頁數有幾頁，是否需要換頁 
    #但韓總實在太多筆了，我們先爬前十頁就好
    url = 'https://www.chinatimes.com/search/韓國瑜?chdtv&page='+str(i+1)
    res = requests.get(url) #取得網頁資料
    soup = BeautifulSoup(res.text, 'lxml') #將網頁資料存到BeautifulSoup4可以讀取的格式
    print(i)
    for context in soup.find_all('div',{'class':'col'}): #從網頁的程式碼中找需要的資料。
        df1 = pd.DataFrame({'title':[context.find('h3',{'class':'title'}).get_text()],
                            'context':[context.find('p',{'class':'intro'}).get_text()],
                            'url':[context.find('a').get('href')]})
        df = pd.concat([df,df1],axis=0)
#         time.sleep(2) #設定停頓時間

0
1
2
3
4
5
6
7
8
9


In [12]:
df.reset_index(inplace=True) #重設index

In [13]:
df.drop([0],axis=0,inplace=True) #把不要的資料欄位移除
df.drop(['index'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
df.to_excel('中時電子報韓國瑜爬蟲結果.xlsx') #輸出資料